# HD Parent Key Exposure: Example (c++)

<br>
<img src="images/hd_parent_exposure.jpg" alt="drawing" style="" width="700px"/>



The above figure briefly recaps how parent private keys can be leaked based on child private keys when deriving non-hardened children.

The goal of this exercise is to demonstrate how this works.

If you need some hints, you can check out this implementation in the `pywallet` library:
https://github.com/ranaroussi/pywallet/blob/468622dcf993a27a5b585289b2724986c02a1fbc/pywallet/utils/bip32.py#L380-L429

In [4]:
# We use this function to call bx from Python, see below for an example
import subprocess
def call(cmd):
    res = subprocess.check_output([cmd], shell=True)
    return res.strip().decode()

### Derive new parent key pair

In [5]:
parent_xpriv = call('bx seed | bx mnemonic-new | bx mnemonic-to-seed | bx hd-new')
parent_xpub = call(f'bx hd-to-public {parent_xpriv}')
print(parent_xpriv)
print(parent_xpub)

parent_priv = call(f'bx hd-to-ec {parent_xpriv}')
print(parent_priv)

xprv9s21ZrQH143K37TEUha1TeQKBdLZiYoMt2gyrMLu5cUazKpY1UwsVVGCvx2fzfXvTaa5JzDbWtXmCmNZ9K1c8NLG2ZrV8dNkWdm3f6r8e9D
xpub661MyMwAqRbcFbXhaj71pnM3jfB481XDFFcaejkWdx1Zs89gZ2G83HagnBq3yJUxTP858J8tA9hUgVqH2Z3mH6tAibibA3uFkXsTTQyfZfD
f0cfd7658dfe19e7cf855ebf727998d7e1a8f3c050576bb29b0016bcdde8e90b


### Derive non-hardenend children

In [9]:
child_xpriv = call(f'bx hd-private --index 0 {parent_xpriv}')
child_priv = call(f'bx hd-to-ec {child_xpriv}')
child_xpub = call(f'bx hd-to-public {child_xpriv}')

print("Exposed material:")
print(f"Parent xpub: {parent_xpub}")
print(f"Child priv: {child_priv}")
print("Child index: 0")

Exposed material:
Parent xpub: xpub661MyMwAqRbcFbXhaj71pnM3jfB481XDFFcaejkWdx1Zs89gZ2G83HagnBq3yJUxTP858J8tA9hUgVqH2Z3mH6tAibibA3uFkXsTTQyfZfD
Child priv: 01ce06974af6b56232c01bf15f1cc98228fdcdad699aa8b9dcc55df5154e5b47
Child index: 0


### Step 1: Extract chaincode from parent xpub

In [10]:
import base58
xpub_bytes = base58.b58decode(parent_xpub)
chaincode = xpub_bytes[13:13+32]
print(list(chaincode))

[105, 205, 144, 79, 94, 145, 161, 221, 2, 16, 144, 70, 198, 153, 189, 13, 234, 28, 112, 5, 148, 19, 13, 1, 31, 245, 64, 48, 232, 4, 156, 159]


### Step2: Compute L256 bits as HMAC-SHA512(Parent Chain Code, Parent Public Key || Child Index)

In [11]:
parent_pub = call(f'bx hd-to-ec {parent_xpub}')
print(parent_pub)
concatenated = parent_pub + "00000000"

024eb7dbdf982285dc382026a48f1c194b2166d5e3d0f7aa65c808a040efa02571


In [13]:
import hmac
import hashlib

l256 = hmac.new(
    chaincode,
    msg=bytes.fromhex(concatenated),
    digestmod=hashlib.sha512
).hexdigest()[:64]

print(l256)

10fe2f31bcf89b7a633abd31eca330a90203b6d3c88bdd430197a5c5079bb37d


### Step 3: Compute Parent private key as child private key - L256

In [14]:
group_order = "fffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f"

group_order_m_l256 = hex(int(group_order, 16) - int(l256, 16))[2:]

print(call(f'bx ec-add-secrets {child_priv} {group_order_m_l256}'))

print(parent_priv)

f0cfd7658dfe19e7cf855ebf727998d926fa16d9a10ecb76db2db82f0db2a3f9
f0cfd7658dfe19e7cf855ebf727998d7e1a8f3c050576bb29b0016bcdde8e90b
